In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [ ]:
acsee = pd.read_csv("CompleteDatasets/necta_acsee_2018.csv")
centersmeta = pickle.load(open('CompleteDatasets/centers_meta_2018.pkl', 'rb'))

In [ ]:
#Any noticeable performance differences between smaller schools and larger schools?

divs = ['I', 'II', 'III', 'IV', '0']
divs_pass = ['I', 'II']

acsee['center'] = acsee['CNO'].apply(lambda x: x.split('/')[0])

def get_region(center, centers_meta):
    try:
        return centers_meta[center]['rankings'].loc[0][1]
    except AttributeError:
        return np.nan

def get_center_size(center, centers_meta):
    try:
        return centers_meta[center]['rankings'].loc[3][1]
    except AttributeError:
        return np.nan

def get_tester_type(center):
    if 'P' in center:
        return 'Private'
    else:
        return 'School'

def get_pass(div):
    if div in divs_pass:
        return 1
    elif div in divs:
        return 0
    else:
        return np.nan

acsee['passed'] = acsee['DIV'].apply(lambda x: get_pass(x))
acsee['region'] = acsee['center'].apply(lambda x: get_region(x, centersmeta))
acsee['centersize'] = acsee['center'].apply(lambda x: get_center_size(x, centersmeta))
acsee['testertype'] = acsee['center'].apply(lambda x: get_tester_type(x))
acsee['centersize'].value_counts() #This is the number of students/candidates within each group

In [ ]:
#This is the % of students in each division
print("Centers with 30 candidates or more")
print(acsee[acsee['DIV'].isin(divs)].groupby('centersize')['DIV'].value_counts()['CENTRE WITH 30 CANDIDATES OR MORE']/acsee[acsee['DIV'].isin(divs)]['centersize'].value_counts()['CENTRE WITH 30 CANDIDATES OR MORE'])
print("\nCenters with less than 30 candidates")
print(acsee[acsee['DIV'].isin(divs)].groupby('centersize')['DIV'].value_counts()['CENTRE WITH LESS THAN 30 CANDIDATES']/acsee[acsee['DIV'].isin(divs)]['centersize'].value_counts()['CENTRE WITH LESS THAN 30 CANDIDATES'])

#The distribution is the same order for each group. DIV II is where most students land, followed by III, I, IV, 0.
#+0.6% difference between divsion 0s. However, outside of a 2% difference between Division IIs, no noticeable diffs.

In [ ]:
#Well that's surprising, why isn't there a noticeable difference between the students from the two types of centers? Let's check to see
#the spread of the number of students for each center.
acsee[acsee['DIV'].isin(divs)].groupby('region').count()['CNO'] #students per region
acsee[acsee['DIV'].isin(divs)].groupby(['center']).count()['CNO'] #students per center
acsee[acsee['DIV'].isin(divs)].groupby(['center', 'centersize']).count()['CNO'] #students per center with centersize

In [ ]:
#Distribution of # of Students at the official size tiers
fig, (ax1, ax2) = plt.subplots(ncols = 2)
print("30 candidates or more - # of students distribution\n", acsee[acsee['DIV'].isin(divs)].groupby(['center', 'centersize']).count()['CNO'][:, 'CENTRE WITH 30 CANDIDATES OR MORE'].describe())
acsee[acsee['DIV'].isin(divs)].groupby(['center', 'centersize']).count()['CNO'][:, 'CENTRE WITH 30 CANDIDATES OR MORE'].plot(kind='hist', ax = ax1,
                                                                                                                            title = '30 or More')

print("Less than 30 candidates - # of students distribution\n", acsee[acsee['DIV'].isin(divs)].groupby(['center', 'centersize']).count()['CNO'][:, 'CENTRE WITH LESS THAN 30 CANDIDATES'].describe())
acsee[acsee['DIV'].isin(divs)].groupby(['center', 'centersize']).count()['CNO'][:, 'CENTRE WITH LESS THAN 30 CANDIDATES'].plot(kind='hist', ax = ax2,
                                                                                                                              title = 'Less than 30')
plt.close()
fig.tight_layout()
fig

In [ ]:
#It seems it might be helpful to disaggregate the 30 or More group of schools
center_students = pd.DataFrame(acsee[acsee['DIV'].isin(divs)].groupby(['center']).count()['CNO']).reset_index()
center_students.rename(columns = {'CNO': 'numstudents'}, inplace=True)
len(center_students[(center_students['numstudents'] > 200) & (center_students['numstudents'] < 300)])

#Most are between 30 and 100 students
#Possible new intervals 30-50, 50-75, 75-100, 100-200, 200-300, 300-900
new_sizes = ['30-50', '50-75', '75-100', '100-200', '200-300', '300-900']
new_sizes_dict = dict.fromkeys(new_sizes)
for k,v in new_sizes_dict.items():
    new_v = (int(k.split('-')[0]), int(k.split('-')[1]))
    new_sizes_dict[k] = new_v

for k,v in new_sizes_dict.items():
    print(len(center_students[(center_students['numstudents'] >= v[0]) & (center_students['numstudents'] < v[1])]))
#Not very scientifically done and groupings aren't as even as possible, but better distribution than 544 vs. 130.
#Let's see how performance compares for this new disaggregation

In [ ]:
#center_students
acsee = acsee.merge(center_students, how = 'left', on='center')
#acsee['new_centersize'] = if numstudents is >= first val and < second val of the value, then return the key
new_sizes_dict['< 30'] = (0, 30)

def get_new_centersize(num, sizesdict):
    #rev sizesdict is tuple: 'interval'
    for k, v in sizesdict.items():
        if num >= v[0] and num < v[1]:
            return k

acsee['new_centersize'] = acsee['numstudents'].apply(lambda x: get_new_centersize(x, new_sizes_dict))
acsee['new_centersize'].unique()

#del acsee['numstudents_x'], acsee['numstudents_y'], acsee['new_centersize']

In [ ]:
list(new_sizes_dict.items())

In [ ]:
#len(acsee[acsee.testertype == 'School'].groupby(['new_centersize', 'center'])['CNO'].count()['< 30']) should be 130

#We've successfully assigned new size groupings. Let's run the analysis again with the new center sizes:

fig, ax = plt.subplots(nrows = 7, figsize = (10,30))

for i in range(len(ax)):
    print(list(new_sizes_dict.items())[i][0], acsee[(acsee['DIV'].isin(divs)) & (acsee['testertype'] == 'School')].groupby(['center', 'new_centersize']).count()['CNO'][:, list(new_sizes_dict.items())[i][0]].describe())
    acsee[(acsee['DIV'].isin(divs)) & (acsee['testertype'] == 'School')].groupby(['center', 'new_centersize']).count()['CNO'][:, list(new_sizes_dict.items())[i][0]].plot(kind='hist', ax = ax[i], title = list(new_sizes_dict.items())[i][0])

plt.close()
fig.tight_layout()
fig.savefig("new_sizes_dist.png")
fig

In [ ]:
#The distributions look better and should allow us to better compare the performance at each size tier
for i in range(7):
    tier = list(new_sizes_dict.items())[i][0]
    print(tier,
          acsee[(acsee['DIV'].isin(divs)) & (acsee['testertype'] == 'School')].groupby(['new_centersize'])['DIV'].value_counts()[tier]/acsee[(acsee['DIV'].isin(divs)) & (acsee['testertype'] == 'School')]['new_centersize'].value_counts()[tier])

In [ ]:
#The new acceptable way to make categorical variables - unused for now, instead can use .reindex method
from pandas.api.types import CategoricalDtype
div_order = ['ABS', '*E', '*R', '*W', '0', 'IV', 'III', 'II', 'I'] #All values must be included in this list, otherwise will be turned to NaN when cat'd
acsee['DIV'] = acsee['DIV'].astype(CategoricalDtype(div_order, ordered=True))

fig, ax = plt.subplots(nrows=7, figsize = (10,20))
for i in range(7):
    tier = ([list(new_sizes_dict.items())[-1]] + (list(new_sizes_dict.items())[0:-1]))[i][0]
    print(tier,
          (acsee[(acsee['DIV'].isin(divs)) & (acsee['testertype'] == 'School')].groupby(['new_centersize'])['DIV'].value_counts()[tier]/acsee[(acsee['DIV'].isin(divs)) & (acsee['testertype'] == 'School')]['new_centersize'].value_counts()[tier]).reindex(divs).plot(kind='bar', ax = ax[i], title = tier, color = 'navy', rot=0))
    


plt.close()
fig.tight_layout()
fig.savefig("new_tiers_div_percentage_dist.png")

#Doing the same as above, but plotting instead, it's easier to see how the trend holds for every size tier except the largest.
#The most promising size tier appears to be the 75-100 group.

In [ ]:
#A more rigorous hypothesis testing may fit here. I use a chi-square test to determine whether a student passing
#is independent of their school's tier size. That is, given two or more school tier sizes are students equally likely
#to pass. Chi-square compares the expected frequencies and observed frequencies to determine whether or not to
#reject the null hypothesis: that there is no difference in passing rates between the school tiers.

from scipy.stats import chisquare

#We need samples of each group

subsamples = []
for i in ['50-75', '300-900']:
    subsample = acsee[(acsee['new_centersize'] == i) & (acsee['testertype'] == 'School') & (acsee['center'] != 'S0485')].sample(frac = 0.09)
    subsamples.append(subsample)
acsee_new_center_sample = pd.concat(subsamples)
print(len(acsee_new_center_sample))

subsamples = []
for i in [e for e in acsee['centersize'].unique() if pd.notnull(e)]:
    subsample = acsee[(acsee['centersize'] == i) & (acsee['testertype'] == 'School')].sample(frac = 0.09)
    subsamples.append(subsample)
acsee_gov_center_sample = pd.concat(subsamples)
print(len(acsee_gov_center_sample))

def generate_chitable(df, which_size = 'centersize'):
    """Return chi square calculation ready table"""
    if which_size == 'centersize':
        observed_passed = df.groupby('centersize')['passed'].sum() #observed passed
        total_candidates = df.groupby('centersize')['passed'].count().rename("Total") #total
        observed_failed = (df.groupby('centersize')['passed'].count() - df.groupby('centersize')['passed'].sum()).rename("failed") #observed not passed
        chi_table = pd.concat([observed_passed, observed_failed, total_candidates], axis = 1)
        chi_table_horizontal_total = chi_table.sum().rename("Total")
        chi_table = chi_table.append(chi_table_horizontal_total)
        chi_table.index.rename("", inplace=True)
    else:
        observed_passed = df.groupby('new_centersize')['passed'].sum() #observed passed
        total_candidates = df.groupby('new_centersize')['passed'].count().rename("Total") #total
        observed_failed = (df.groupby('new_centersize')['passed'].count() - df.groupby('new_centersize')['passed'].sum()).rename("failed") #observed not passed
        chi_table = pd.concat([observed_passed, observed_failed, total_candidates], axis = 1)
        chi_table_horizontal_total = chi_table.sum().rename("Total")
        chi_table = chi_table.append(chi_table_horizontal_total)
        chi_table.index.rename("", inplace=True)
    
    return chi_table


def get_expected_values(chitable):
    """The return value is in the pattern of passed(row1), failed(row1), passed(row2), failed(row2)"""
    e_values = []
    for i in range(len(chitable)-1):
        passed_e = (chitable.iloc[i, 2] * chitable.iloc[len(chitable)-1,0])/chitable['Total']['Total']
        e_values.append(passed_e)
        failed_e = (chitable.iloc[i, 2] * chitable.iloc[len(chitable)-1,1])/chitable['Total']['Total']
        e_values.append(failed_e)
    return np.array(e_values)

def get_observed_values(chitable):
    """The return value is in the pattern of passed(row1), failed(row1), passed(row2), failed(row2)"""
    o_values = []
    for i in range(len(chitable)-1):
        passed_o = chitable.iloc[i,0]
        o_values.append(passed_o)
        failed_o = chitable.iloc[i,1]
        o_values.append(failed_o)
    return np.array(o_values)


#I find no significant differences between the official government groupings of the school, but the new groupings show
#there is a significant difference p < 0.05. Not surprisingly, the driver of this difference appears to be the 300-900 group.
#However, after removing the outlier schoool S0485, < 30 shows no significant difference in performance with 300-900 (now 600),
#but the 75-100 group still has significantly different passing rates. As do the other mid-tier schools.
print(chisquare(get_observed_values(generate_chitable(acsee_new_center_sample, "new_centersize")), get_expected_values(generate_chitable(acsee_new_center_sample, "new_centersize"))))

print(chisquare(get_observed_values(generate_chitable(acsee_gov_center_sample)), get_expected_values(generate_chitable(acsee_gov_center_sample))))

In [ ]:
#The p-value changes with every chi-square test. Let's look at the distribution of P-values if we ran multiple times.
#I would run the test and often get p-values that are statistically significant, and others that are not.

#For reference, new sizes: ['50-75', '< 30', '100-200', '75-100', '30-50', '200-300', '300-900', None]
#Article: http://varianceexplained.org/statistics/interpreting-pvalue-histogram/
p_vals = []
rounds = 0

while rounds < 100:
    subsamples = []
    for i in ['< 30', '300-900']:
        subsample = acsee[(acsee['new_centersize'] == i) & (acsee['testertype'] == 'School') & (acsee['center'] != 'S0485')].sample(frac = 0.09)
        subsamples.append(subsample)
    acsee_new_center_sample = pd.concat(subsamples)

    x, p = chisquare(get_observed_values(generate_chitable(acsee_new_center_sample, "new_centersize")), get_expected_values(generate_chitable(acsee_new_center_sample, "new_centersize")))
    p_vals.append(p)
    rounds += 1

plt.hist(p_vals)
plt.show()
#50-75, < 30 conservative p-values
#100-200, < 30 conservative
#75-100, < 30 bimodal
#200-300, < 30 conservative
#300-900, < 30 weird/bimodal
#50-75, 75-100 bimodal?
#50-75, 100-200 conservative
#50-75, 200-300 anti-conservative
#50-75, 300-900 anti-conservative
#75-100, 100-200 anti-conservative
#75-100, 200-300 anti-conservative
#75-100, 300-900 anti-conservative
#100-200, 200-300 anti-conservative
#100-200, 300-900 anti-conservative
#200-300, 300-900 conservative

#The anti-conservative ones (all have a very high % of alternative hypotheses):
#50-75, 200-300 anti-conservative
#50-75, 300-900 anti-conservative
#75-100, 100-200 anti-conservative
#75-100, 200-300 anti-conservative
#75-100, 300-900 anti-conservative
#100-200, 200-300 anti-conservative
#100-200, 300-900 anti-conservative

#From above, data shows that smaller tiers perform better on average. This appears to be statistically significant
#for the comparisons that are exhibiting an anti-conservative distribution here.


In [ ]:
#For the regular centersize tiers

p_vals = []
rounds = 0
while rounds < 500:
    subsamples = []
    for i in [e for e in acsee['centersize'].unique() if pd.notnull(e)]:
        subsample = acsee[(acsee['centersize'] == i) & (acsee['testertype'] == 'School') & (acsee['center'] != 'S0485')].sample(frac = 0.09)
        subsamples.append(subsample)
    acsee_gov_center_sample = pd.concat(subsamples)

    x, p = chisquare(get_observed_values(generate_chitable(acsee_gov_center_sample)), get_expected_values(generate_chitable(acsee_gov_center_sample)))
    p_vals.append(p)
    rounds += 1

plt.hist(p_vals)
plt.show()
#The distribution looks like something (an assumption) is wrong with the test (according to the blog post).
#And there likely is - the More than 30 or more group was had a left-skewed distribution.

In [ ]:
#If 75-100 seems to be doing well, does 75-100 tier schools correlate to any region?
#Mtwara, the highest performing region, only had 5% of its students in the 75-100 size tier schools.
#Kaskazini Pemba, the third lowest performing region, had the highest share of its students in schools in that tier.
#Not too surprisingly, having a greater concentration of the best performing tier doesn't necessarily translate to performance.
numstudentstier75 = acsee[(acsee['new_centersize'] == '75-100') & (acsee['DIV'].isin(divs))]['region'].value_counts()
(numstudentstier75/acsee[(acsee['DIV'].isin(divs))]['region'].value_counts()).sort_values(0)

In [ ]:
#When comparing to the concentration of _schools_ (not students) in that size, a similar story emerges.
numschoolstier75 = acsee[(acsee['new_centersize'] == '75-100') & (acsee['DIV'].isin(divs))].groupby('region')['center'].unique().apply(lambda x: len(x))
(numschoolstier75/acsee[(acsee['DIV'].isin(divs))].groupby('region')['center'].unique().apply(lambda x: len(x))).sort_values(0)

In [ ]:
#Total number of schools per each region - pretty large difference here in terms of the number of schools to manage.
#However, Zanzibar schools and Pemba are an anomaly. Relatively fewer number of schools, but not high performing.
#Likely because they are islands and don't receive as much resources in the way of education
#but maybe they should, given that they're tourism hotspots. Similarly, I'd expect Katavi to perform better.
acsee[(acsee['DIV'].isin(divs))].groupby('region')['center'].unique().apply(lambda x: len(x)).sort_values(0)

In [ ]:
#For the top 3 regions, find their size tier distributions (number of students at each tier)
top3numstudents = pd.DataFrame(acsee[(acsee['DIV'].isin(divs))].groupby(['region'])['new_centersize'].value_counts()[['GEITA', 'MTWARA', 'LINDI']] )
top3numstudents.columns = ['numstudents']
top3numstudents.reset_index(inplace=True)
top3numstudents.set_index(['region', 'new_centersize'], inplace=True)

#In order to divide just the three regions (or other select regions), the indices of the dividend/divisor need to match.
#This is simple for a single index situation, but trickier for multi-index especially since the divisor is not a scalar -
#the divisor is the total number of students in the region in question. We elect to have a multi-index so that we can have
#clear size tier labels for the region without assuming. So we have to replicate the dividend dataframe, add the multi-indices as
#columns of the new dataframe (totalstudents), reset so that we can map total values (which are scalars) based on the region,
#set the multi-index for the new dataframe (totalstudents), then divide the two series.

totalstudents = pd.DataFrame(index = top3numstudents.index, columns = ['totalstudents']).reset_index()
studenttotals = dict(list(acsee[(acsee['DIV'].isin(divs))]['region'].value_counts()[list(top3numstudents.index.levels[0])].items()))
totalstudents['totalstudents'] = totalstudents['region'].apply(lambda x: studenttotals[x] ) #if region is x, put y values
totalstudents.set_index(['region', 'new_centersize'], inplace=True)
top3numstudents['numstudents']/totalstudents['totalstudents']

#Once again, if 75-100 seems to appear to be the _most_ optimal school size tier, a small proportion of the
#students attend these schools in the highest performing regions. Suggesting that their success is not driven by tier 
#size alone. 

#A regression idea: passing % = size_tier_dummies + region_control

In [ ]:
#If Kaskazini Pemba's 75-100 tier schools didn't carry it to the finish line, how did
#the rest of the other regions' 75-100 tier schools do?
numstudentstier75
tier75passing = acsee[(acsee['DIV'].isin(divs_pass)) & (acsee['testertype'] == 'School') & (acsee['new_centersize'] == '75-100')].groupby('region').count()['CNO']

(tier75passing/numstudentstier75).sort_values(0, ascending=False)

#Top performing regions had high passing rates among students in the 75-100 tier schools.
#If this is the spread for this tier, what about the other tiers?

In [ ]:
region_size_pass = acsee[(acsee['DIV'].isin(divs_pass)) & (acsee['testertype'] == 'School') ].groupby(['region', 'new_centersize']).count()['CNO']
region_size_total = acsee[(acsee['DIV'].isin(divs)) & (acsee['testertype'] == 'School')].groupby(['region', 'new_centersize']).count()['CNO']

#(region_size_pass/region_size_total).to_frame().sort_values(['region', 'CNO']).to_csv("region_size_pass_rate.csv")

(region_size_pass/region_size_total).to_frame().sort_values(['region', 'CNO'])

In [ ]:
#To easily compare different regions/center sizes, use pd.IndexSlice
idx = pd.IndexSlice
(region_size_pass/region_size_total).to_frame().sort_values(['region', 'CNO']).loc[idx[['DAR ES SALAAM', 'GEITA',
                                                                                       'LINDI', 'MTWARA',
                                                                                       'KASKAZINI PEMBA'], :], :]

In [ ]:
#This is hard to see. A line plot with center sizes in order, and passing rates in y-axis to see trend would help
from pandas.api.types import CategoricalDtype
size_order = ['< 30', '30-50', '50-75', '75-100', '100-200', '200-300', '300-900'] #All values must be included in this list, otherwise will be turned to NaN when cat'd
acsee['new_centersize'] = acsee['new_centersize'].astype(CategoricalDtype(size_order, ordered=True))

In [ ]:
#GROUP BY PLOT DOES WHAT I'VE BEEN MANUALLY DOING HERE WHAT
#https://scentellegher.github.io/programming/2017/07/15/pandas-groupby-multiple-columns-plot.html

fig, ax = plt.subplots(figsize=(20,10))
unstacked = (region_size_pass/region_size_total).to_frame().groupby(['new_centersize', 'region']).mean()['CNO'].unstack().reset_index()
unstacked['new_centersize'] = unstacked['new_centersize'].astype(CategoricalDtype(size_order, ordered=True))
unstacked.sort_values('new_centersize').plot('new_centersize', ax=ax, legend=False, title='Candidate passing rates as school tier size increases')
plt.close()
ax.set_ylabel('Passing Rate')
ax.set_xlabel('School Tier Sizes: < 30 to 300-900 Candidates')
fig.savefig('passing_rates_size_tier_increase.jpg')
fig

#This is providing another visual look at how passing rates change for candidates depending on which tier their school is in
#The peak appears to be in the 75-100 tiers, and a downward slope from there
#But I think the surprising information here is that there is a general upward trend from < 30 to 75-100, meaning
#less may not always be more. And we can definitely disaggregate this further to look at regional zones as well
#but overall, having too many students and too few appears to not be beneficial. (Each line is a region)

In [ ]:
#A follow up twitter question: https://twitter.com/BLACKSTEMUSA/status/1072129663114362880

fig, ax = plt.subplots(figsize = (20, 30))
unstacked = (region_size_pass/region_size_total).to_frame().groupby(['new_centersize', 'region']).mean()['CNO'].unstack().reset_index()
unstacked['new_centersize'] = unstacked['new_centersize'].astype(CategoricalDtype(size_order, ordered=True))
axis = unstacked.sort_values('new_centersize').plot('new_centersize', subplots = True,
                                             ax=ax, legend=False, sharey = True, color='navy',
                                            layout = (8, 4), xticks = [0,1,2,3,4,5,6]) #xticks here uses the index, default use_index=True, because < causes error. This works because new_centersize is sorted

plt.close()

subplot_titles = list(unstacked.columns)[1:]
title_num = 0
for row in axis:
    for plot in row:
        if title_num >= len(subplot_titles):
            continue
        plot.set_xlabel('School Tier Sizes')
        plot.set_title(subplot_titles[title_num])
        title_num += 1
fig.tight_layout(rect=[0, 0, 1, 0.95])
fig.suptitle("Candidate Passing Rates as School Tier Size Increases", fontsize = 16)
#fig.savefig("passing_rate_tier_size_2.jpg", dpi = 600)
fig